In [7]:
%env AWS-PROFILE=just-my-links

import boto3
import requests
import json


env: AWS-PROFILE=just-my-links


In [8]:
cloudformation_outputs_res = !aws cloudformation describe-stacks --stack-name just-my-links-dev --query 'Stacks[0].Outputs'
cloudformation_outputs = {x['OutputKey']: x['OutputValue'] for x in json.loads(''.join(cloudformation_outputs_res))}
application_bucket_name = cloudformation_outputs['ApplicationBucketName']
document_store_uri = cloudformation_outputs['DocumentStorageHttpApiUrl']
store_document_arn = cloudformation_outputs['StoreDocumentFunctionArn']
index_documents_arn = cloudformation_outputs['IndexDocumentsFunctionArn']
auth_token_secret_arn = cloudformation_outputs['AuthTokenSecretName']

secrets_client = boto3.client('secretsmanager')
auth_token_response = secrets_client.get_secret_value(SecretId=auth_token_secret_arn)
auth_token = auth_token_response['SecretString']

# Utility Functions

These can be called further down to do things

In [1]:
def send_test_to_document_store(document_url= "https://example.com/test-article"):
    import urllib.parse

    endpoint_url = f"{document_store_uri}/document"

    # Prepare multipart form data
    files = {
        'document': ('document.html',  "<html><body><h1>Test Article</h1><p>This is a test article content.</p></body></html>", 'text/html')
    }

    response = requests.put(
        endpoint_url,
        params = {'url': document_url},
        headers={"Authorization": f"Bearer {auth_token}"},
        files=files
    )

    return (response.status_code, response.text)

See the logs

In [3]:
def tail_store_document_logs():
    res = !aws logs tail /aws/lambda/just-my-links--store-document--dev
    return res

Deploy changes to both lambda services

In [ ]:
!scripts/deploy-document-storage-service.py

In [ ]:
!scripts/deploy-indexing-service.py

Turn on debug logging with

In [5]:
def toggle_store_document_debug_logs(turn_on=True):
    function_name = 'just-my-links--store-document--dev'
    lambda_client = boto3.client('lambda')

    response = lambda_client.get_function_configuration(FunctionName=function_name)

    current_variables = {**response.get('Environment', {}).get('Variables', {}), "LOG_LEVEL": ("DEBUG" if turn_on else "INFO")}

    lambda_client.update_function_configuration(FunctionName=function_name, Environment={'Variables': current_variables})


# Current work


In [19]:
import urllib.parse

# Test document storage with multipart form data
document_url = "https://example.com/test-article"
document_content = "<html><body><h1>Test Article</h1><p>This is a test article content.</p></body></html>"

endpoint_url = f"http://127.0.0.1:8080/document"

# Prepare multipart form data
files = {
    'document': ('document.html', document_content, 'text/html')
}

response = requests.put(
    endpoint_url,
    params = {'url': document_url},
    headers={"Authorization": f"Bearer {auth_token}"},
    files=files
)

Running the above to a simple server I wrote shows what gets sent. It looks pretty reasonable

In [9]:
send_test_to_document_store()

(400, '{"error":"Invalid Content-Type header"}')

In [10]:
tail_store_document_logs()

['2025-06-07T19:59:59.016000+00:00 2025/06/07/[$LATEST]b3dcf326f2cd47fd97384567c718fa08 START RequestId: e62e02fc-3543-4162-9673-8916d74d25a7 Version: $LATEST',
 '2025-06-07T19:59:59.017000+00:00 2025/06/07/[$LATEST]b3dcf326f2cd47fd97384567c718fa08 {"level":"DEBUG","location":"lambda_handler:421","message":"Lambda handler invoked","timestamp":"2025-06-07 19:59:59,017+0000","service":"service_undefined","cold_start":true,"function_name":"just-my-links--store-document--dev","function_memory_size":"1024","function_arn":"arn:aws:lambda:us-east-1:145216492484:function:just-my-links--store-document--dev","function_request_id":"e62e02fc-3543-4162-9673-8916d74d25a7","correlation_id":"Lz0Jsho2oAMEY1g=","event_type":"unknown","path":"unknown","resource":"unknown","request_context":{"accountId":"145216492484","apiId":"r4259u5i5d","domainName":"r4259u5i5d.execute-api.us-east-1.amazonaws.com","domainPrefix":"r4259u5i5d","http":{"method":"PUT","path":"/dev/document","protocol":"HTTP/1.1","sourceIp":